In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os
import time
import pickle as pkl
import argparse

from t2_functions import partition_ids, partition_ids2, find_independent_set

import sys

sys.path.append(os.path.join(os.getcwd(), "task_2_sybilattack/"))

from endpoints.requests import sybil, sybil_reset

print(os.getcwd())
dataset = torch.load("data/SybilAttack.pt")

ids = np.array(dataset.ids)
binned_ids = partition_ids(ids, main_bin_num=10)

#binned_ids2 = partition_ids2(ids, dataset.imgs, main_bin_num=10)

task = 'affine'

sybil_reset(home_or_defense='home', binary_or_affine=task)
sybil_reset(home_or_defense='defense', binary_or_affine=task)
time.sleep(10)

/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack


ConnectionError: HTTPConnectionPool(host='34.71.138.79', port=9090): Max retries exceeded with url: /sybil/affine/reset/home (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x76f147301250>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [39]:
for i in range(2, 10):
    print(f'main loop: {i}')
    sybil_reset(home_or_defense='defense', binary_or_affine=task)
    time.sleep(10)
    ids = binned_ids[i]
    print(len(ids))
    B_reps = sybil(ids=ids,
                home_or_defense='defense',
                binary_or_affine=task)
    '''
    B_indep, indexes, success = find_independent_set(B_reps, k=200)
    print(f"success: {success}")

    ids_train = ids[indexes]
    mask = np.ones(len(ids), dtype=bool)
    mask[indexes] = False
    ids_test = ids[mask]'''

    B_reps = np.array(B_reps)
    B_train_reps = B_reps[:200]
    B_test_reps = B_reps[200:]

    A_train_reps = sybil(ids=ids[:200],
                    home_or_defense='home',
                    binary_or_affine=task)
    print(f"A train reps: {len(A_train_reps)}")

    folder = os.path.join('data', 'submit', task, str(1), f'partition_{i}')
    os.makedirs(folder, exist_ok=True)

    with open(f'{folder}/A_train', 'wb') as f:
        pkl.dump(A_train_reps, f)
    with open(f'{folder}/B_train', 'wb') as f:
        pkl.dump(B_train_reps, f)
    with open(f'{folder}/B_test', 'wb') as f:
        pkl.dump(B_test_reps, f)
    with open(f'{folder}/ids_train', 'wb') as f:
        pkl.dump(ids[:200], f)
    with open(f'{folder}/ids_test', 'wb') as f:
        pkl.dump(ids[200:], f)


main loop: 2


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))